## File includes codes to run MLE on the 4 models (multiple sampling strategies used for the coarse grid search)

### G1

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import cho_factor, cho_solve
from scipy.optimize import minimize
from numpy.polynomial import chebyshev
from scipy.special import gegenbauer

_EPS = 1e-12
d = 2  # dimension of the grid
p = 1  # positive odd integer

# ============================================================
# PSD helper
# ============================================================
def make_psd(C, eps=1e-9):
    C = 0.5 * (C + C.T)
    w, V = np.linalg.eigh(C)
    w = np.maximum(w, eps)
    return (V * w) @ V.T


# ============================================================
# Even / odd kernels
# ============================================================
def G_even(h, a):
    return np.exp(-(h @ h) / (a * a + _EPS))


def G_odd(h, a, u0):
    hnorm = (h @ h) ** 0.5
    if hnorm == 0:
        return 0
    else:
        if d == 2:
            # NOTE: this matches your original usage
            Tp = chebyshev.Chebyshev.basis(p)#chebyshev.Chebyshev(p)
            return ((-1) ** ((p - 1) / 2)) * (a ** (1 - p)) * Tp((h @ u0) / hnorm) * np.exp(
                -(h @ h) / (2.0 * a + _EPS)
            )
        
        #else:
        #use the formula for d>2


# ============================================================
# C2 block and full covariance
# ============================================================
def C2_block(h, a11, a22, a12, rho, s1, s2, u0):
    C11 = s1 * s1 * G_even(h, a11)
    C22 = s2 * s2 * G_even(h, a22)

    C12 = rho * s1 * s2 * G_odd(h, a12, u0)
    C21 = -C12

    return np.array([[C11, C12],
                     [C21, C22]])


def build_cov_C2(loc, a11, a22, a12, rho, s1, s2, u0):
    N = len(loc)
    C = np.zeros((2 * N, 2 * N))
    for i in range(N):
        for j in range(N):
            h = loc[j] - loc[i]
            block = C2_block(h, a11, a22, a12, rho, s1, s2, u0)
            C[2 * i:2 * i + 2, 2 * j:2 * j + 2] = block
    return make_psd(C)


# ============================================================
# NLL with multiple time replicates (full parameter vector)
# ============================================================
def total_nll_full_params(params, loc, y_list, s1, s2, u0):
    a11, a22, a12, rho = params

    # Basic constraints
    if a11 <= 0 or a22 <= 0 or a12 <= 0 or abs(rho) >= 1:
        return 1e20

    C = build_cov_C2(loc, a11, a22, a12, rho, s1, s2, u0)
    if np.isnan(C).any() or np.isinf(C).any():
        return 1e20

    try:
        cF = cho_factor(C, check_finite=False)
        logdet = 2.0 * np.sum(np.log(np.diag(cF[0])))
    except Exception:
        return 1e20

    m = C.shape[0]        # 2N
    R = len(y_list)       # number of time replicates

    quad_sum = 0.0
    try:
        for y in y_list:
            alpha = cho_solve(cF, y, check_finite=False)
            quad_sum += float(y @ alpha)
    except Exception:
        return 1e20

    nll = 0.5 * (quad_sum + R * logdet + R * m * np.log(2.0 * np.pi))
    return nll


# ============================================================
# Joint MLE with grid-search-based initialization
# ============================================================
def coordinate_mle_C2_multi(loc, y_list, s1, s2, u0, init):
    """
    Joint MLE of (a11, a22, a12_even, a12_odd, theta, rho) with:
      1) pseudo-grid search over bounds to find a good starting point
      2) L-BFGS-B optimization from that starting point

    'init' is still accepted and included as one of the candidate starting points.
    """
    # Bounds (Adjust accordingly)
    bounds = [
        (50.0, 4000.0),      # a11
        (50.0, 4000.0),      # a22
        (50.0, 4000.0),      # a12
        (-0.99, 0.99)        # rho
    ]

    # ---- 1) Pseudo-grid search with logging ----
    n_grid = 7           # ~7 values per parameter across its bounds
    n_candidates = 200   # number of random combinations from that grid

    rng = np.random.default_rng(0)

    grid_vals = []
    for (low, high) in bounds:
        grid_vals.append(np.linspace(low, high, n_grid))

    # Start with the user-provided init as a candidate (if valid)
    best_params = np.array(init, dtype=float)
    best_nll = total_nll_full_params(best_params, loc, y_list, s1, s2, u0)

    print("Starting grid search for initial parameters...")
    print(f"Initial candidate from 'init': NLL = {best_nll:.3f}")

    for cand_i in range(n_candidates):
        idxs = rng.integers(0, n_grid, size=len(bounds))
        candidate = np.array([grid_vals[k][idxs[k]] for k in range(len(bounds))], dtype=float)
        nll_val = total_nll_full_params(candidate, loc, y_list, s1, s2, u0)

        if nll_val < best_nll:
            best_nll = nll_val
            best_params = candidate

        # Simple progress logging
        if (cand_i + 1) % 50 == 0 or cand_i == 0:
            print(
                f"Grid search candidate {cand_i + 1}/{n_candidates} "
                f"| current best NLL = {best_nll:.3f}"
            )

    print("Best grid-search params:", best_params)
    print("Best grid-search NLL:", best_nll)

    # ---- 2) Local optimization from best grid-search point ----
    print("\nStarting local optimization from best grid-search point...")
    res = minimize(
        lambda params: total_nll_full_params(params, loc, y_list, s1, s2, u0),
        x0=best_params,
        bounds=bounds,
        method="L-BFGS-B",
        options={'maxiter': 100}
    )

    print("Optimization success:", res.success)
    print("Message:", res.message)
    print("Final params:", res.x)
    print("Final NLL:", res.fun)

    a11_hat, a22_hat, a12_hat, rho_hat = res.x
    return a11_hat, a22_hat, a12_hat, rho_hat


# ============================================================
# Main
# ============================================================
print("Loading multi-time U/V dataset...")
df = pd.read_csv("uv_wind_multi_for_C2.csv", parse_dates=["datetime"])
print("Total rows:", len(df))

# 1) Build spatial grid from the FIRST time slice
times = np.sort(df["datetime"].unique())
t0 = times[0]
df0 = df[df["datetime"] == t0].copy()

# Sort by (lat, lon) to get consistent ordering
df0 = df0.sort_values(["latitude", "longitude"]).reset_index(drop=True)

lat0 = df0["latitude"].to_numpy()
lon0 = df0["longitude"].to_numpy()

# Convert deg -> km 
lat_mean = np.mean(lat0)
x_km = 111.0 * lon0 * np.cos(np.radians(lat_mean))
y_km = 111.0 * lat0
loc = np.column_stack([x_km, y_km])
N = loc.shape[0]

print("Spatial grid size N =", N)

# 2) Build list of y_t vectors across time replicates
y_list = []
max_reps = 20  # upper limit on number of time replicates

for tt in times:
    if len(y_list) >= max_reps:
        break

    df_t = df[df["datetime"] == tt].copy()
    # Ensure same grid/order as df0 by merging
    df_t = df_t.merge(
        df0[["latitude", "longitude"]],
        on=["latitude", "longitude"],
        how="right",
        sort=False
    )

    # If any missing, skip this time
    if df_t["u_std"].isna().any() or df_t["v_std"].isna().any():
        continue

    df_t = df_t.sort_values(["latitude", "longitude"]).reset_index(drop=True)

    Y1 = df_t["u_std"].to_numpy()
    Y2 = df_t["v_std"].to_numpy()

    if len(Y1) != N or len(Y2) != N:
        continue

    y_t = np.zeros(2 * N)
    y_t[0::2] = Y1
    y_t[1::2] = Y2

    y_list.append(y_t)

R = len(y_list)
print("Number of time replicates used:", R)

if R == 0:
    raise RuntimeError("No complete time replicate found.")

# 3) Fit C2 with replicates
s1 = s2 = 1.0
u0 = np.array([1.0, 0.0])  # east–west

print("\nFitting C2 model with multiple time replicates...\n")

init_params = (200.0, 200.0, 200.0, 0.5)

a11_hat, a22_hat, a12_hat, rho_hat = coordinate_mle_C2_multi(
    loc, y_list, s1, s2, u0, init_params
)

print("\n===== FINAL ESTIMATES (C2, multi-time U/V) =====")
print("a11       =", a11_hat)
print("a22       =", a22_hat)
print("a12  =", a12_hat)
print("rho       =", rho_hat)

# 4) Model fit statistics: full NLL, AIC, BIC
final_params = (a11_hat, a22_hat, a12_hat, rho_hat)
NLL = total_nll_full_params(final_params, loc, y_list, s1, s2, u0)

m = 2 * N
k = 6
n_obs = R * m

AIC = 2 * k + 2 * NLL
BIC = k * np.log(n_obs) + 2 * NLL

print("\n===== MODEL FIT (C2, multi-time U/V) =====")
print("NLL =", NLL)
print("AIC =", AIC)
print("BIC =", BIC)
print("N   =", N, "locations; R =", R, "replicates; total obs =", n_obs)


## G2

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import cho_factor, cho_solve
from scipy.optimize import minimize
from numpy.polynomial import chebyshev
from scipy.special import gegenbauer

_EPS = 1e-12
d = 2  # dimension of the grid
p = 1  # positive odd integer

# ============================================================
# PSD helper
# ============================================================
def make_psd(C, eps=1e-9):
    C = 0.5 * (C + C.T)
    w, V = np.linalg.eigh(C)
    w = np.maximum(w, eps)
    return (V * w) @ V.T


# ============================================================
# Even / odd kernels
# ============================================================
def G_even(h, a):
    return np.exp(-(h @ h) / (a * a + _EPS))


def G_odd(h, a, u0):
    hnorm = (h @ h) ** 0.5
    if hnorm == 0:
        return 0
    else:
        if d == 2:
            # NOTE: this matches your original usage
            Tp = chebyshev.Chebyshev.basis(p)#chebyshev.Chebyshev(p)
            return ((-1) ** ((p - 1) / 2)) * (a ** (1 - p)) * Tp((h @ u0) / hnorm) * np.exp(
                -(h @ h) / (2.0 * a + _EPS)
            )

        #else:
            #Cp = gegenbauer(p, d / 2 - 1)
            #Use the formula with the Genenbaur polynomial


# ============================================================
# C2 block and full covariance
# ============================================================
def C2_block(h, a11, a22, a12_even, a12_odd, theta, rho, s1, s2, u0):
    C11 = s1 * s1 * G_even(h, a11)
    C22 = s2 * s2 * G_even(h, a22)

    even_part = G_even(h, a12_even)
    odd_part = G_odd(h, a12_odd, u0)

    mix_plus = (np.cos(theta))**2 * even_part + (np.sin(theta))**2 * odd_part
    mix_minus = (np.cos(theta))**2 * even_part - (np.sin(theta))**2 * odd_part

    C12 = rho * s1 * s2 * mix_plus
    C21 = rho * s1 * s2 * mix_minus

    return np.array([[C11, C12],
                     [C21, C22]])


def build_cov_C2(loc, a11, a22, a12_even, a12_odd, theta, rho, s1, s2, u0):
    N = len(loc)
    C = np.zeros((2 * N, 2 * N))
    for i in range(N):
        for j in range(N):
            h = loc[j] - loc[i]
            block = C2_block(h, a11, a22, a12_even, a12_odd, theta, rho, s1, s2, u0)
            C[2 * i:2 * i + 2, 2 * j:2 * j + 2] = block
    return make_psd(C)


# ============================================================
# NLL with multiple time replicates (full parameter vector)
# ============================================================
def total_nll_full_params(params, loc, y_list, s1, s2, u0):
    a11, a22, a12_even, a12_odd, theta, rho = params

    # Basic constraints
    if a11 <= 0 or a22 <= 0 or a12_even <= 0 or a12_odd <= 0 or abs(rho) >= 1:
        return 1e20

    C = build_cov_C2(loc, a11, a22, a12_even, a12_odd, theta, rho, s1, s2, u0)
    if np.isnan(C).any() or np.isinf(C).any():
        return 1e20

    try:
        cF = cho_factor(C, check_finite=False)
        logdet = 2.0 * np.sum(np.log(np.diag(cF[0])))
    except Exception:
        return 1e20

    m = C.shape[0]        # 2N
    R = len(y_list)       # number of time replicates

    quad_sum = 0.0
    try:
        for y in y_list:
            alpha = cho_solve(cF, y, check_finite=False)
            quad_sum += float(y @ alpha)
    except Exception:
        return 1e20

    nll = 0.5 * (quad_sum + R * logdet + R * m * np.log(2.0 * np.pi))
    return nll


# ============================================================
# Joint MLE with grid-search-based initialization
# ============================================================
def coordinate_mle_C2_multi(loc, y_list, s1, s2, u0, init):
    """
    Joint MLE of (a11, a22, a12_even, a12_odd, theta, rho) with:
      1) pseudo-grid search over bounds to find a good starting point
      2) L-BFGS-B optimization from that starting point

    'init' is still accepted and included as one of the candidate starting points.
    """
    # Bounds (same as before)
    bounds = [
        (50.0, 4000.0),      # a11
        (50.0, 4000.0),      # a22
        (50.0, 4000.0),      # a12_even
        (50.0, 4000.0),      # a12_odd
        (0.0, 2.0 * np.pi),  # theta
        (-0.99, 0.99)        # rho
    ]

    # ---- 1) Pseudo-grid search with logging ----
    n_grid = 7           # ~7 values per parameter across its bounds
    n_candidates = 200   # number of random combinations from that grid

    rng = np.random.default_rng(0)

    grid_vals = []
    for (low, high) in bounds:
        grid_vals.append(np.linspace(low, high, n_grid))

    # Start with the user-provided init as a candidate (if valid)
    best_params = np.array(init, dtype=float)
    best_nll = total_nll_full_params(best_params, loc, y_list, s1, s2, u0)

    print("Starting grid search for initial parameters...")
    print(f"Initial candidate from 'init': NLL = {best_nll:.3f}")

    for cand_i in range(n_candidates):
        idxs = rng.integers(0, n_grid, size=len(bounds))
        candidate = np.array([grid_vals[k][idxs[k]] for k in range(len(bounds))], dtype=float)
        nll_val = total_nll_full_params(candidate, loc, y_list, s1, s2, u0)

        if nll_val < best_nll:
            best_nll = nll_val
            best_params = candidate

        # Simple progress logging
        if (cand_i + 1) % 50 == 0 or cand_i == 0:
            print(
                f"Grid search candidate {cand_i + 1}/{n_candidates} "
                f"| current best NLL = {best_nll:.3f}"
            )

    print("Best grid-search params:", best_params)
    print("Best grid-search NLL:", best_nll)

    # ---- 2) Local optimization from best grid-search point ----
    print("\nStarting local optimization from best grid-search point...")
    res = minimize(
        lambda params: total_nll_full_params(params, loc, y_list, s1, s2, u0),
        x0=best_params,
        bounds=bounds,
        method="L-BFGS-B",
        options={'maxiter': 500}
    )

    print("Optimization success:", res.success)
    print("Message:", res.message)
    print("Final params:", res.x)
    print("Final NLL:", res.fun)

    a11_hat, a22_hat, a12_even_hat, a12_odd_hat, theta_hat, rho_hat = res.x
    return a11_hat, a22_hat, a12_even_hat, a12_odd_hat, theta_hat, rho_hat


# ============================================================
# Main
# ============================================================
print("Loading multi-time U/V dataset...")
df = pd.read_csv("Data.csv", parse_dates=["datetime"])#load the relevant data
print("Total rows:", len(df))

# 1) Build spatial grid from the FIRST time slice
times = np.sort(df["datetime"].unique())
t0 = times[0]
df0 = df[df["datetime"] == t0].copy()

# Sort by (lat, lon) to get consistent ordering
df0 = df0.sort_values(["latitude", "longitude"]).reset_index(drop=True)

lat0 = df0["latitude"].to_numpy()
lon0 = df0["longitude"].to_numpy()

# Convert deg -> km
lat_mean = np.mean(lat0)
x_km = 111.0 * lon0 * np.cos(np.radians(lat_mean))
y_km = 111.0 * lat0
loc = np.column_stack([x_km, y_km])
N = loc.shape[0]

print("Spatial grid size N =", N)

# 2) Build list of y_t vectors across time replicates
y_list = []
max_reps = 20  # upper limit on number of time replicates

for tt in times:
    if len(y_list) >= max_reps:
        break

    df_t = df[df["datetime"] == tt].copy()
    # Ensure same grid/order as df0 by merging
    df_t = df_t.merge(
        df0[["latitude", "longitude"]],
        on=["latitude", "longitude"],
        how="right",
        sort=False
    )

    # If any missing, skip this time
    if df_t["u_std"].isna().any() or df_t["v_std"].isna().any():
        continue

    df_t = df_t.sort_values(["latitude", "longitude"]).reset_index(drop=True)

    Y1 = df_t["u_std"].to_numpy()
    Y2 = df_t["v_std"].to_numpy()

    if len(Y1) != N or len(Y2) != N:
        continue

    y_t = np.zeros(2 * N)
    y_t[0::2] = Y1
    y_t[1::2] = Y2

    y_list.append(y_t)

R = len(y_list)
print("Number of time replicates used:", R)

if R == 0:
    raise RuntimeError("No complete time replicate found.")

# 3) Fit C2 with replicates
s1 = s2 = 1.0
u0 = np.array([1.0, 0.0])  # east–west

print("\nFitting C2 model with multiple time replicates...\n")

init_params = (200.0, 200.0, 200.0, 200.0, np.pi, 0.5)

a11_hat, a22_hat, a12_even_hat, a12_odd_hat, theta_hat, rho_hat = coordinate_mle_C2_multi(
    loc, y_list, s1, s2, u0, init_params
)

print("\n===== FINAL ESTIMATES (C2, multi-time U/V) =====")
print("a11       =", a11_hat)
print("a22       =", a22_hat)
print("a12_even  =", a12_even_hat)
print("a12_odd   =", a12_odd_hat)
print("theta     =", theta_hat)
print("rho       =", rho_hat)

# 4) Model fit statistics: full NLL, AIC, BIC
final_params = (a11_hat, a22_hat, a12_even_hat, a12_odd_hat, theta_hat, rho_hat)
NLL = total_nll_full_params(final_params, loc, y_list, s1, s2, u0)

m = 2 * N
k = 6
n_obs = R * m

AIC = 2 * k + 2 * NLL
BIC = k * np.log(n_obs) + 2 * NLL

print("\n===== MODEL FIT (C2, multi-time U/V) =====")
print("NLL =", NLL)
print("AIC =", AIC)
print("BIC =", BIC)
print("N   =", N, "locations; R =", R, "replicates; total obs =", n_obs)


## M1

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import cho_factor, cho_solve
from scipy.optimize import minimize
from numpy.polynomial import chebyshev
from scipy.special import gamma, kv
from scipy.spatial.distance import pdist, squareform
_EPS = 1e-12
d = 2  
p = 3   # odd integer


# ============================================================
# PSD helper
# ============================================================
def make_psd(C, eps=1e-9):
    C = 0.5 * (C + C.T)
    w, V = np.linalg.eigh(C)
    w = np.maximum(w, eps)
    return (V * w) @ V.T


# ============================================================
# Even / odd kernels  (Matérn-like)
# ============================================================
def G_even(h, a, nu):
    hnorm = np.sqrt(h @ h)
    if hnorm == 0:
        return 1.0

    ah = a * hnorm
    kv_val = kv(nu, ah)
    if not np.isfinite(kv_val) or kv_val <= 0:
        return 0.0

    val = (2 ** (1 - nu)) / gamma(nu) * (ah ** nu) * kv_val
    return float(val)


def G_odd(h, a, u0, nu):
    hnorm = np.sqrt(h @ h)
    if hnorm == 0:
        return 0.0

    ah = a * hnorm
    kv_val = kv(p - nu, ah)
    if not np.isfinite(kv_val) or kv_val <= 0:
        return 0.0

    # correct Chebyshev polynomial T_p
    Tp = chebyshev.Chebyshev.basis(p)
    cosang = (h @ u0) / hnorm
    cosang = np.clip(cosang, -1, 1)

    Tval = Tp(cosang)

    val = ((-1) ** ((p - 1) / 2)) * (ah ** nu) * Tval * kv_val
    return float(val)


# ============================================================
# Covariance
# ============================================================
def C2_block(h, a11, a22, a12, nu11, nu22, nu12, rho, s1, s2, u0):
    C11 = s1 * s1 * G_even(h, a11, nu11)
    C22 = s2 * s2 * G_even(h, a22, nu22)
    C12 = rho * s1 * s2 * G_odd(h, a12, u0, nu12)
    C21 = -C12
    return np.array([[C11, C12], [C21, C22]])


def build_cov_C2(loc, a11, a22, a12, nu11, nu22, nu12, rho, s1, s2, u0):
    N = len(loc)
    C = np.zeros((2 * N, 2 * N))

    for i in range(N):
        for j in range(N):
            h = loc[j] - loc[i]
            C[2*i:2*i+2, 2*j:2*j+2] = C2_block(h, a11, a22, a12, nu11, nu22, nu12, rho, s1, s2, u0)

    return make_psd(C)


# ============================================================
# NLL
# ============================================================
def total_nll_full_params(params, loc, y_list, s1, s2, u0):
    a11, a22, a12, nu11, nu22, nu12, rho = params

    if a11 <= 0 or a22 <= 0 or a12 <= 0 or abs(rho) >= 1:
        return 1e20

    # IMPORTANT: enforce ν constraints
    if nu11 <= 0 or nu22 <= 0 or nu12 <= 0:
        return 1e20

    C = build_cov_C2(loc, a11, a22, a12, nu11, nu22, nu12, rho, s1, s2, u0)
    if np.isnan(C).any() or np.isinf(C).any():
        return 1e20

    try:
        cF = cho_factor(C, check_finite=False)
        logdet = 2 * np.sum(np.log(np.diag(cF[0])))
    except:
        return 1e20

    m = C.shape[0]
    R = len(y_list)

    quad = 0.0
    try:
        for y in y_list:
            alpha = cho_solve(cF, y, check_finite=False)
            quad += y @ alpha
    except:
        return 1e20

    return 0.5 * (quad + R * logdet + R * m * np.log(2 * np.pi))


# ============================================================
# Joint MLE (grid search + L-BFGS-B)
# ============================================================
def coordinate_mle_C2_multi(loc, y_list, s1, s2, u0, init):
    bounds = [
        (50.0, 4000.0),   # a11
        (50.0, 4000.0),   # a22
        (50.0, 4000.0),   # a12
        (0.1, 10.0),      # nu11
        (0.1, 10.0),      # nu22
        (0.1, 10.0),      # nu12
        (-0.99, 0.99)     # rho
    ]

    # ---- simple grid search ----
    n_grid = 20
    n_candidates = 100
    rng = np.random.default_rng(0)

    grid_vals = [np.linspace(l, u, n_grid) for (l, u) in bounds]

    best_params = np.array(init, float)
    best_nll = total_nll_full_params(best_params, loc, y_list, s1, s2, u0)

    print("Grid search start, initial NLL =", best_nll)

    for i in range(n_candidates):
        idx = rng.integers(0, n_grid, len(bounds))
        cand = np.array([grid_vals[k][idx[k]] for k in range(len(bounds))])
        nll = total_nll_full_params(cand, loc, y_list, s1, s2, u0)

        if nll < best_nll:
            best_nll = nll
            best_params = cand

        if (i + 1) % 50 == 0:
            print(f"Grid {i+1}/{n_candidates}, best NLL = {best_nll}")

    print("Best grid params:", best_params)

    # ---- L-BFGS-B ----
    res = minimize(
        lambda p: total_nll_full_params(p, loc, y_list, s1, s2, u0),
        best_params,
        bounds=bounds,
        method="L-BFGS-B",
        options={"maxiter": 500}
    )

    print("Optimizer:", res.message)
    print("Final params:", res.x)
    print("Final NLL:", res.fun)

    return res.x


# ============================================================
# Main
# ============================================================
print("Loading dataset...")
df = pd.read_csv("Data.csv", parse_dates=["datetime"])

times = np.sort(df["datetime"].unique())
df0 = df[df["datetime"] == times[0]].sort_values(["latitude","longitude"]).reset_index(drop=True)

lat0 = df0["latitude"].values
lon0 = df0["longitude"].values

lat_mean = np.mean(lat0)
x_km = 111 * lon0 * np.cos(np.radians(lat_mean))
y_km = 111 * lat0
loc = np.column_stack([x_km, y_km])
##
D = squareform(pdist(loc))          # all pairwise distances in km
np.fill_diagonal(D, np.inf)
nn_dists = D.min(axis=1)            # nearest neighbor distance for each location
dist_scale = nn_dists.mean()        # average nn distance (in km)
loc = loc / dist_scale

##
N = len(loc)

# Build y vectors
y_list = []
for tt in times[:20]:
    df_t = df[df["datetime"] == tt].copy()
    df_t = df_t.merge(df0[["latitude","longitude"]], on=["latitude","longitude"], how="right")
    if df_t["u_std"].isna().any() or df_t["v_std"].isna().any():
        continue
    df_t = df_t.sort_values(["latitude","longitude"]).reset_index(drop=True)

    y = np.zeros(2*N)
    y[0::2] = df_t["u_std"].values
    y[1::2] = df_t["v_std"].values
    y_list.append(y)

R = len(y_list)
print("Replicates:", R)

s1 = s2 = 1.0
u0 = np.array([1.0, 0.0])

init_params = (800,800,800, 1,1,1, 0.3)
params = coordinate_mle_C2_multi(loc, y_list, s1, s2, u0, init_params)

a11_hat, a22_hat, a12_hat, nu11_hat, nu22_hat, nu12_hat, rho_hat = params

print("\nFinal parameters:")
print(params)

NLL = total_nll_full_params(params, loc, y_list, s1, s2, u0)

k = 7
m = 2*N
n_obs = R * m

AIC = 2*k + 2*NLL
BIC = k*np.log(n_obs) + 2*NLL

print("\nNLL =", NLL)
print("AIC =", AIC)
print("BIC =", BIC)


## M2

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import cho_factor, cho_solve
from scipy.optimize import minimize
from numpy.polynomial import chebyshev
from scipy.special import gamma, kv
from scipy.spatial.distance import pdist, squareform

_EPS = 1e-12
d = 2  # dimension of the grid
p = 5  # positive odd integer

# ============================================================
# PSD helper
# ============================================================
def make_psd(C, eps=1e-6):
    """
    Symmetrize and add a small nugget for numerical stability.
    """
    C = 0.5 * (C + C.T)
    C = C + eps * np.eye(C.shape[0])
    return C


# ============================================================
# Even / odd kernels (Matérn-type)
# ============================================================
def G_even(h, a, nu):
    hnorm = (h @ h) ** 0.5
    if hnorm == 0:
        return 1.0
    else:
        x = a * hnorm
        val = ((2 ** (1 - nu)) / gamma(nu)) * (x ** nu) * kv(nu, x)
        # Safety: if kv underflowed to 0 or NaN, just treat as 0 correlation.
        if not np.isfinite(val):
            return 0.0
        return float(val)


def G_odd(h, a, u0, nu):
    hnorm = (h @ h) ** 0.5
    if hnorm == 0:
        return 0.0
    else:
        if d == 2:
            Tp = chebyshev.Chebyshev.basis(p)
            x = a * hnorm
            val = (
                ((-1) ** ((p - 1) / 2))
                * (x ** nu)
                * Tp((h @ u0) / hnorm)
                * kv(p - nu, x)
            )
            if not np.isfinite(val):
                return 0.0
            return float(val)
        # For d != 2 you'd implement the Gegenbauer-based version here.
        raise NotImplementedError("G_odd currently only implemented for d=2.")


# ============================================================
# C2 block and full covariance
# ============================================================
def C2_block(h, a11, a22, a12, a12e, nu11, nu22, nu12, nu12e, rho, s1, s2, u0,theta):
    C11 = s1 * s1 * G_even(h, a11, nu11)
    C22 = s2 * s2 * G_even(h, a22, nu22)

    even_part = G_even(h, a12,nu12)
    odd_part = G_odd(h, a12e, u0, nu12e)

    mix_plus = np.cos(theta) * even_part + np.sin(theta) * odd_part
    mix_minus = np.cos(theta) * even_part - np.sin(theta) * odd_part

    C12 = rho * s1 * s2 * mix_plus
    C21 = rho * s1 * s2 * mix_minus

    return np.array([[C11, C12],
                     [C21, C22]])


def build_cov_C2(loc, a11, a22, a12, a12e, nu11, nu22, nu12, nu12e, rho, s1, s2, u0,theta):
    N = len(loc)
    C = np.zeros((2 * N, 2 * N))
    for i in range(N):
        for j in range(N):
            h = loc[j] - loc[i]
            block = C2_block(h, a11, a22, a12, a12e, nu11, nu22, nu12, nu12e, rho, s1, s2, u0,theta)
            C[2 * i:2 * i + 2, 2 * j:2 * j + 2] = block
    return make_psd(C)


# ============================================================
# NLL with multiple time replicates (full parameter vector)
# ============================================================
def total_nll_full_params(params, loc, y_list, s1, s2, u0):
    """
    params = (a11, a22, a12, nu11, nu22, nu12, rho)
    """
    a11, a22, a12, a12e, nu11, nu22, nu12e, nu12, rho, theta = params

    # Basic constraints
    if a11 < 0 or a22 < 0 or a12 < 0 or a12e < 0 or abs(rho) > 1:
        return 1e20

    C = build_cov_C2(loc, a11, a22, a12, a12e, nu11, nu22, nu12, nu12e, rho, s1, s2, u0, theta)
    if np.isnan(C).any() or np.isinf(C).any():
        return 1e20

    try:
        cF = cho_factor(C, check_finite=False)
        logdet = 2.0 * np.sum(np.log(np.diag(cF[0])))
    except Exception:
        return 1e20

    m = C.shape[0]        # 2N
    R = len(y_list)       # number of time replicates

    quad_sum = 0.0
    try:
        for y in y_list:
            alpha = cho_solve(cF, y, check_finite=False)
            quad_sum += float(y @ alpha)
    except Exception:
        return 1e20

    nll = 0.5 * (quad_sum + R * logdet + R * m * np.log(2.0 * np.pi))
    return nll


# ============================================================
# Coordinate-wise grid search + L-BFGS-B refinement
# ============================================================
def coordinate_mle_C2_multi(loc, y_list, s1, s2, u0, init):
    """
    1) Coordinate-wise grid search:
       - 7 parameters: (a11, a22, a12, nu11, nu22, nu12, rho)
       - Each has 5 grid values (linspace within bounds)
       - At each step, pick best value for one parameter while holding others fixed
       - Repeat sweeps until no improvement or max_sweeps reached

    2) Continuous L-BFGS-B optimization from the best grid solution.
    """

    # Bounds for all 7 parameters
    bounds = [
        (3, 6000.0),   # a11
        (3, 6000.0),   # a22
        (3, 6000.0),   # a12
        (3, 6000.0),   # a12e
        (0.5, 50.0),     # nu11
        (0.5, 50.0),     # nu22
        (0.5, 50.0),     # nu12
        (0.5, 50.0),     # nu12e
        (-1, 1),    # rho
        (0.01, 2.0*np.pi-0.01)     # theta
    ]

    param_names = ["a11", "a22", "a12", "a12e", "nu11", "nu22", "nu12", "nu12e", "rho", "theta"]

    # Build n-point grids for each parameter
    n_grid = 100
    grid_vals = []
    for (low, high) in bounds:
        grid_vals.append(np.linspace(low, high, n_grid))

    # Start from init, but clamp to bounds and snap to nearest grid point
    current_params = np.array(init, dtype=float)
    for k in range(len(bounds)):
        low, high = bounds[k]
        # clamp
        current_params[k] = min(max(current_params[k], low), high)
        # snap to nearest grid value
        gv = grid_vals[k]
        idx_nearest = np.argmin(np.abs(gv - current_params[k]))
        current_params[k] = gv[idx_nearest]

    current_nll = total_nll_full_params(current_params, loc, y_list, s1, s2, u0)
    print("Initial (snapped) params:", dict(zip(param_names, current_params)))
    print("Initial NLL:", current_nll)

    max_sweeps = 5
    tol_improve = 1e-6

    for sweep in range(1, max_sweeps + 1):
        print(f"\n===== Coordinate sweep {sweep}/{max_sweeps} =====")
        improved_this_sweep = False

        for k in range(len(bounds)):
            print(f"  Optimizing parameter {k} ({param_names[k]})...")

            best_val_k = current_params[k]
            best_nll_k = current_nll

            # Try all grid values for parameter k
            for candidate_val in grid_vals[k]:
                trial_params = current_params.copy()
                trial_params[k] = candidate_val
                nll_val = total_nll_full_params(trial_params, loc, y_list, s1, s2, u0)

                if nll_val < best_nll_k - tol_improve:
                    best_nll_k = nll_val
                    best_val_k = candidate_val

            # Update if improved
            if best_nll_k < current_nll - tol_improve:
                print(f"    Updated {param_names[k]}: {current_params[k]} -> {best_val_k}, "
                      f"NLL {current_nll:.6f} -> {best_nll_k:.6f}")
                current_params[k] = best_val_k
                current_nll = best_nll_k
                improved_this_sweep = True
            else:
                print(f"    No improvement for {param_names[k]} (kept {current_params[k]})")

        print("  End of sweep", sweep, "| current params:",
              dict(zip(param_names, current_params)),
              "| current NLL:", current_nll)

        if not improved_this_sweep:
            print("\nNo improvement in this sweep; stopping coordinate search.")
            break

    # ---------------------------------------------
    # After grid search: continuous L-BFGS-B step
    # ---------------------------------------------
    print("\nCoordinate grid search complete.")
    print("Best grid parameters:", dict(zip(param_names, current_params)))
    print("Grid-based NLL:", current_nll)

    print("\nStarting continuous optimization (L-BFGS-B)...")

    bounds_full = bounds  # same as above

    res = minimize(
        lambda params: total_nll_full_params(params, loc, y_list, s1, s2, u0),
        x0=current_params,
        bounds=bounds_full,
        method="L-BFGS-B",
        options={'maxiter': 200, 'ftol': 1e-12}
    )

    print("\nL-BFGS-B results:")
    print("Success:", res.success)
    print("Message:", res.message)
    print("Final parameters:", res.x)
    print("Final NLL:", res.fun)

    return tuple(res.x)


# ============================================================
# Main
# ============================================================
print("Loading multi-time U/V dataset...")
df = pd.read_csv("Data.csv", parse_dates=["datetime"])
print("Total rows:", len(df))

# 1) Build spatial grid from the FIRST time slice
times = np.sort(df["datetime"].unique())
t0 = times[0]
df0 = df[df["datetime"] == t0].copy()

# Sort by (lat, lon) to get consistent ordering
df0 = df0.sort_values(["latitude", "longitude"]).reset_index(drop=True)

lat0 = df0["latitude"].to_numpy()
lon0 = df0["longitude"].to_numpy()

# Convert deg -> km
lat_mean = np.mean(lat0)
x_km = 111.0 * lon0 * np.cos(np.radians(lat_mean))
y_km = 111.0 * lat0
loc = np.column_stack([x_km, y_km])
N = loc.shape[0]

print("Spatial grid size N =", N)

# ---- Rescale to dimensionless coordinates ----
D = squareform(pdist(loc))
np.fill_diagonal(D, np.inf)
nn_dists = D.min(axis=1)
dist_scale = nn_dists.mean()  # typical nearest-neighbor spacing

print("Average nearest-neighbor distance (km) =", dist_scale)

loc_scaled = loc / dist_scale
print("Using rescaled locations (dimensionless) with typical spacing ~1.")

# 2) Build list of y_t vectors across time replicates
y_list = []
max_reps = 20  # upper limit on number of time replicates

for tt in times:
    if len(y_list) >= max_reps:
        break

    df_t = df[df["datetime"] == tt].copy()
    # Ensure same grid/order as df0 by merging
    df_t = df_t.merge(
        df0[["latitude", "longitude"]],
        on=["latitude", "longitude"],
        how="right",
        sort=False
    )

    # If any missing, skip this time
    if df_t["u_std"].isna().any() or df_t["v_std"].isna().any():
        continue

    df_t = df_t.sort_values(["latitude", "longitude"]).reset_index(drop=True)

    Y1 = df_t["u_std"].to_numpy()
    Y2 = df_t["v_std"].to_numpy()

    if len(Y1) != N or len(Y2) != N:
        continue

    y_t = np.zeros(2 * N)
    y_t[0::2] = Y1
    y_t[1::2] = Y2

    y_list.append(y_t)

R = len(y_list)
print("Number of time replicates used:", R)

if R == 0:
    raise RuntimeError("No complete time replicate found.")

# 3) Fit C2 with replicates, using rescaled loc
s1 = s2 = 1.0
u0 = np.array([1.0, 0.0])  # east–west

print("\nFitting C2 model with multiple time replicates (coordinate-wise grid + L-BFGS-B)...\n")

init_params = (3.437, 3.572, 475.57, 0.0001, 29.9986, 29.9188, 1.5, 1.578, 0.26, np.pi/2)

a11_hat, a22_hat, a12_hat, a12e_hat, nu11_hat, nu22_hat, nu12_hat,  nu12e_hat, rho_hat, theta_hat = coordinate_mle_C2_multi(
    loc_scaled, y_list, s1, s2, u0, init_params
)

print("\n===== FINAL ESTIMATES (C2, multi-time U/V) =====")
print("a11       =", a11_hat)
print("a22       =", a22_hat)
print("a12       =", a12_hat)
print("a12e       =", a12e_hat)
print("nu11      =", nu11_hat)
print("nu22      =", nu22_hat)
print("nu12      =", nu12_hat)
print("nu12e      =", nu12e_hat)
print("rho       =", rho_hat)
print("theta       =", theta_hat)

# 4) Model fit statistics: full NLL, AIC, BIC
final_params = (a11_hat, a22_hat, a12_hat, a12e_hat, nu11_hat, nu22_hat, nu12_hat, nu12e_hat, rho_hat, theta_hat)
NLL = total_nll_full_params(final_params, loc_scaled, y_list, s1, s2, u0)

m = 2 * N
k = 10  # a11, a22, a12, nu11, nu22, nu12, rho
n_obs = R * m

AIC = 2 * k + 2 * NLL
BIC = k * np.log(n_obs) + 2 * NLL

print("\n===== MODEL FIT (C2, multi-time U/V) =====")
print("NLL =", NLL)
print("AIC =", AIC)
print("BIC =", BIC)
print("N   =", N, "locations; R =", R, "replicates; total obs =", n_obs)
